In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient

In [3]:
report_files = glob.glob("sample_reports/*.txt")

In [4]:
reports = []
report_ids = []
for report_file in report_files:
    report_ids.append(os.path.basename(report_file))
    with open(report_file, 'r') as f:
        reports.append(f.read())

In [5]:
prompt = Prompt('demo_prompt_no_CoT.toml')

In [6]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

In [7]:
engine = RadPrompter(
    client=client, 
    prompt=prompt,
    concurrency=12,
    hide_blocks=True,
    output_directory="output"
)

In [8]:
sample = [{'report': sample_report} for sample_report in reports]

In [9]:
sample = [{'report': sample_report, 'report_id': report_id} for sample_report, report_id in zip(reports, report_ids)]

In [10]:
results = engine(sample)

Processing items: 100%|██████████| 3/3 [00:01<00:00,  1.95it/s]


In [11]:
engine.save_log("demo_no_CoT.log")

In [12]:
# Flatten the nested dictionary structure
flattened_data = []
for row in results:
    row_dict = {}
    for d in row:
        key, value = list(d.items())[0]
        row_dict[key] = value[0]
    flattened_data.append(row_dict)

# Create the DataFrame
results_df = pd.DataFrame(flattened_data)

In [13]:
results_df

,report_id,Pulmonary Embolism,Left,Right,Acute,Chronic,RightHeartStrain,PulmonaryArteryHypertension
0,sample_report_1.txt,Present,Yes,Yes,Yes,No,Yes,No
1,sample_report_2.txt,Present,Yes,Yes,No,No,No,No
2,sample_report_3.txt,Present,Yes,Yes,No,Yes,No,Yes
